In [1]:
import acquire
import explore
import model

import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

# Big Question: Why are our customers churning?

## Hypotheses:
1. Tenure Cohorts
    - $H_0$: There are no tenure cohort with a higher propensity to churn than others
    - $H_a$: There is a cohort with a higher propensity to churn
2. Features w/ higher churn rate
    - $H_0$: There are no features with a higher churn rate than others
    - $H_a$: There is at least one feature with a higher churn rate
3. Price threshold with higher churn rate
    - $H_0$: There is no price threshold for specific services where the likelihood of churn increases
    - $H_a$: Higher price services are more likeley to churn
4. Month-to-month customers after the 12th month anD 1-year contract customers after the 12th month, are those rates comparable?
    - $H_0$: Month-to-month customers and 1-year contract customers are equally likely to churn after the 12th month
    - $H_a$: Month-to-month customers are more likely to churn after the 12th month than 1-year contract customers

## Acquire and Prep Data
 - See acquire.py and prepare.py for details
 - Produces train, valid & test .csv files

In [2]:
train = pd.read_csv('train.csv')
valid = pd.read_csv('valid.csv')
test = pd.read_csv('test.csv')

## Explore Train Data

1. Tenure Cohorts
    - $H_0$: There are no tenure cohort with a higher propensity to churn than others
    - $H_a$: There is a cohort with a higher propensity to churn

2. Features w/ higher churn rate
    - $H_0$: There are no features with a higher churn rate than others
    - $H_a$: There is at least one feature with a higher churn rate

3. Price threshold with higher churn rate
    - $H_0$: There is no price threshold for specific services where the likelihood of churn increases
    - $H_a$: Higher price services are more likeley to churn

4. Month-to-month customers after the 12th month anD 1-year contract customers after the 12th month, are those rates comparable?
    - $H_0$: Month-to-month customers and 1-year contract customers are equally likely to churn after the 12th month
    - $H_a$: Month-to-month customers are more likely to churn after the 12th month than 1-year contract customers

## Create Predictions Using Decision Tree Model

> During the modeling phase, we found that not all the features were as important to the model as others, however, the overall performance of the model was not significantly improved by dropping them - it actually dropped a small amount. Therefore, this model will include all the features, so as to get more nuance.

> Additionally, the model's performance was not significantly enhanced by scaling the features, so this model will be built with the original features.

> Finally, we will us the Decision Tree classifier to model this dataset, because the performance on the validation dataset was similar to the performance of the other models, but the output of the decision tree is more interpretable.

In [3]:
X_train = train.drop(columns=['churn', 'Unnamed: 0', 'customer_id', 'tenure_nml', 'monthly_charges_nml', 'total_charges_nml', 'churn_enc'])
X_valid = valid.drop(columns=['churn', 'Unnamed: 0', 'customer_id', 'tenure_nml', 'monthly_charges_nml', 'total_charges_nml', 'churn_enc'])
X_test = test.drop(columns=['churn', 'Unnamed: 0', 'customer_id', 'tenure_nml', 'monthly_charges_nml', 'total_charges_nml', 'churn_enc'])

y_train = train.churn
y_valid = valid.churn
y_test = test.churn

In [4]:
# Make the model
DT_model = model.model_maker(X_train, y_train, 'DT', 143)
DT_preds, DT_probs = model.model_predictor(X_train, DT_model)
DT_conf, DT_report = model.model_evaluator(y_train, DT_preds)
print (DT_report)

              precision    recall  f1-score   support

          No       0.81      0.94      0.87      3320
         Yes       0.70      0.40      0.51      1187

    accuracy                           0.80      4507
   macro avg       0.76      0.67      0.69      4507
weighted avg       0.78      0.80      0.78      4507



In [5]:
# Test the model's performance on the validation set
DTv_preds, DTv_probs = model.model_predictor(X_valid, DT_model)
DTv_conf, DTv_report = model.model_evaluator(y_valid, DTv_preds)
print (DTv_report)

              precision    recall  f1-score   support

          No       0.81      0.95      0.87       837
         Yes       0.72      0.34      0.46       290

    accuracy                           0.80      1127
   macro avg       0.76      0.65      0.67      1127
weighted avg       0.78      0.80      0.77      1127



In [6]:
# Test the model's performance on the test set
DTt_preds, DTt_probs = model.model_predictor(X_test, DT_model)
DTt_conf, DTt_report = model.model_evaluator(y_test, DTt_preds)
print (DTt_report)

              precision    recall  f1-score   support

          No       0.79      0.94      0.86      1017
         Yes       0.69      0.36      0.47       392

    accuracy                           0.78      1409
   macro avg       0.74      0.65      0.67      1409
weighted avg       0.76      0.78      0.75      1409



In [7]:
train_probs = pd.DataFrame(DT_probs)
valid_probs = pd.DataFrame(DTv_probs)
test_probs = pd.DataFrame(DTt_probs)

train_preds = pd.DataFrame(DT_preds)
valid_preds = pd.DataFrame(DTv_preds)
test_preds = pd.DataFrame(DTt_preds)

train_preds

,0
0,No
1,Yes
2,No
3,No
4,Yes
...,...
4502,No
4503,Yes
4504,No
4505,No


In [8]:
train_probs = train_probs.rename(columns={1: 'probabblah'})
valid_probs = valid_probs.rename(columns={1: 'probabblah'})
test_probs = test_probs.rename(columns={1: 'probabblah'})

train_preds = train_preds.rename(columns={0: 'pred'})
valid_preds = valid_preds.rename(columns={0: 'pred'})
test_preds = test_preds.rename(columns={0: 'pred'})

In [9]:
# Produce a .csv with the customer_id, probability of churn, and the prediction of churn (1=churn, 0=not_churn).
results_train = pd.DataFrame(train.customer_id)
results_valid = pd.DataFrame(valid.customer_id)
results_test = pd.DataFrame(test.customer_id)

results_train['Probability_churn'] = train_probs['probabblah']
results_train['Prediction'] = train_preds['pred']

results_valid['Probability_churn'] = valid_probs['probabblah']
results_valid['Prediction'] = valid_preds['pred']

results_test['Probability_churn'] = test_probs['probabblah']
results_test['Prediction'] = test_preds['pred']

In [10]:
results_train = results_train.append(results_valid)
results_train = results_train.append(results_test)
results_train

,customer_id,Probability_churn,Prediction
0,5044-LRQAQ,0.500,No
1,2397-BRLOM,0.703,Yes
2,3932-CMDTD,0.164,No
3,4990-ALDGW,0.024,No
4,3422-GALYP,0.703,Yes
...,...,...,...
1404,3229-USWAR,0.164,No
1405,8260-NGFNY,0.593,Yes
1406,5334-JLAXU,0.239,No
1407,2674-MIAHT,0.703,Yes


(4507, 32)